In [1]:
import cvxpy as cp 
import numpy as np 
import itertools as it 
import matplotlib.pyplot as plt 
import math

In [2]:
def calculate_average_dist(grid_coord, bus_coords):
    x1 = grid_coord[0]
    y1 = grid_coord[1]
    avg_dist = 0
     
 
    for coord in bus_coords: 
        x2 = coord[0]
        y2 = coord[1]
        sub_x = x1 - x2
        sub_y = y1 - y2
        distance = math.sqrt(sub_x*sub_x + sub_y*sub_y) 
        avg_dist += distance
    
    avg_dist = avg_dist / len(bus_coords)
    
    return(avg_dist) 


In [3]:
def calc_distance(p1, p2): 
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    sub_x = x1 - x2
    sub_y = y1 - y2
    distance = math.sqrt(sub_x*sub_x + sub_y*sub_y)
    return(distance)

In [4]:
#get latitude and longitude values for defined area - use to make "grid"  
lat = np.linspace(40.0321515, 39.9538968, num = 16) #16 because thats how many grid boxes across
long = np.linspace(-83.0521853, -82.8597894, num = 8) #8 because thats how many grid boxes down
 
#get coordinates for each grid box 
grid_coords = []
for cord in it.product(lat, long): #use cartesian product to get each point
    grid_coords.append(cord)
    

#get coordinates for bus stops 

bus_coords = []


# manually add coordinates for 12 bus stops chosen based on lat and long 
bus_coords.append((40.00636499999999, -83.00921 ))
bus_coords.append((39.989904,-82.913984, ))
bus_coords.append((40.029009,-83.042526))
bus_coords.append((40.025763,-82.981144))
bus_coords.append((40.018846,-82.981144))
bus_coords.append((39.96993399999999,-82.93334))
bus_coords.append((39.97490699999999,-82.88498))
bus_coords.append((39.991938,-82.94982399999999)) 
bus_coords.append((39.9742879,-83.02435940000001)) 
bus_coords.append((39.97284510000001,-82.9806979)) 
bus_coords.append((40.001937,-82.979916)) 
bus_coords.append((40.021141,-82.928016)) 

 

In [5]:
#get the average distance array set up for all the points in the grid  
avg_distances = [] 
for grid_coord in grid_coords: 
        dist =  calculate_average_dist(grid_coord, bus_coords)
        avg_distances.append(dist)
        

In [6]:
def get_obj_func(x, average_distances):
    total_sum = 0
    for i in range(0, 128): 
        total_sum += x[i]*average_distances[i]
    return total_sum

def constraints_val(x):
    sum_val = 0
    for i in range(0, 128): 
        sum_val += x[i]
    return sum_val 

In [7]:
#start optimization problem 

x = cp.Variable(128, boolean = True) # vector variable representing if bank falls in this box or not 
                                        # - 128 because (16*8) gives us number of grid boxes

 
obj_func =  get_obj_func(x, avg_distances)

constraints = []
constraints.append(constraints_val(x) == 3)
constraints.append(x[62] == 0) #prevents food bank from being in Airport 
#whole box is water, industrial site, etc. -> cant add food bank anywhere there 
constraints.append(x[97] == 0)
constraints.append(x[98] == 0)
constraints.append(x[99] == 0)

 
problem = cp.Problem(cp.Minimize(obj_func), constraints)
 
problem.solve(solver=cp.GUROBI,verbose = True)
 
    
print("x = ")
print(x.value)


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 29 08:36:45 PM: Your problem has 128 variables, 5 constraints, and 0 parameters.
(CVXPY) Nov 29 08:36:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 29 08:36:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 29 08:36:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 29 08:36:45 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Nov 29 08:36:45 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuff